El archivo homicidios viene con dos hojas de excel que son victimas y hechos, abro cada una en un notebook diferente para modificar y tranformar los valores que se necesite

La información que contiene la hoja de datos hechos es hechos de transito con vehículos en movimiento 

In [ ]:
pip install geopy

In [93]:
import json
import pandas as pd
import ast
from datetime import datetime, time
from geopy.geocoders import Nominatim

In [94]:
# Abro libro hechos del archivo homicidios y lo dejo como dataframe de csv

hechos_original = r"C:\Users\Usuario\Henry\PI2_DA\Consigna\hechos_original.csv"


In [95]:
df_hechos = pd.read_csv(hechos_original)

In [ ]:
df_hechos

In [ ]:
df_hechos.head()

Se puede ver que este conjunto de datos cuenta con 21 columnas reterentes a las caracteristicas del evento como la fecha el numero de victimas, el lugar, y la ubicación que se podría normalizar de mejor forma. inicialmente voy a dejar una columna con la fecha y la hora para borrar la columnas que contienen esta información

Creo función para verificar datos
Funcion verificar_tipos_datos

La función verificar_tipo_datos toma un DataFrame de Pandas code argumento y realiza un análisis detallado 
de cada columna. Genera  el nombre de la columna, el tipo de datos, 
el porcentaje de valores no nulos, el porcentaje de valores nulos y la cantidad de valores 
nulos para cada columna. Muestra las columnas con valores nulos ordenadas de mayor a menor según el porcentaje de valores nulos.
El resultado es un DataFrame que proporciona una visión detallada de la calidad de los datos en el 
DataFrame original.

In [98]:
def verificar_tipo_datos(df):
# Genero Diccionario vació para almacenar información sobre cada columna
    mi_dict = {"nombre_campo": [], "tipo_datos": [], "no_nulos_%": [], "nulos_%": [], "nulos": []}
    
# Iterar sobre  las columnas del DataFrame df
    for columna in df.columns:
        # Calcular el porcentaje de valores no nulos en la columna actual
        porcentaje_no_nulos = (df[columna].count() / len(df)) * 100
        
# Agregar información de mi_dict
        mi_dict["nombre_campo"].append(columna)
        mi_dict["tipo_datos"].append(df[columna].apply(type).unique())
        mi_dict["no_nulos_%"].append(round(porcentaje_no_nulos, 2))
        mi_dict["nulos_%"].append(round(100 - porcentaje_no_nulos, 2))
        mi_dict["nulos"].append(df[columna].isnull().sum())
        
# Crear un DataFrame con la información recopilada
    df_info = pd.DataFrame(mi_dict)
    
# Calcular el total de valores nulos en el DataFrame
    total_nulos = df.isnull().sum().sum()
    
# Imprimir el total de valores nulos
    print(f'Total de valores nulos en el DataFrame: {total_nulos}')
    
# Mostrar las columnas con valores nulos y sus respectivos porcentajes (ordenadas de mayor a menor)
    columnas_con_nulos = df_info[df_info["nulos"] > 0][["nombre_campo", "nulos_%"]].sort_values(by="nulos_%", ascending=False)
    if not columnas_con_nulos.empty:
        print("\nColumnas con valores nulos y sus porcentajes:")
        print(columnas_con_nulos)
        
# Devolver el DataFrame con la información de cada columna

    return df_info

In [ ]:
verificar_tipo_datos(df_hechos)


In [100]:
df_hechos.size
# es corto

14616

In [ ]:
# La totalidad de nulos fueron 747 que es el   5.11% que es bastante aceptable
# Así que borro columna de altura que no sirve para nada por que ya tengo coordenadas

# Cruce y  direcccion y borro borro na por que deprondto me sirve la calle

In [101]:
df_hechos = df_hechos.drop('Altura', axis=1)

In [102]:
df_hechos['Dirección Normalizada'].dropna(inplace=True)

In [103]:
df_hechos['Cruce'].dropna(inplace=True)

In [ ]:
df_hechos.head()

In [105]:
# Borro columna HH que ya esta en hora

df_hechos = df_hechos.drop(columns=['HH'])


In [106]:
df_hechos.size

13224

In [107]:
# Me aseguro que toda la columna de fecha tenga el mismo formato YY-MM-DD 

df_hechos['FECHA'] = pd.to_datetime(df_hechos['FECHA'], format='%Y-%m-%d')


In [108]:
# Dado que esa columna ya tiene la info de toda la fecha borro columnas de año, mes y dia

columnas_borrar = ['AAAA', 'MM', 'DD']

df_hechos = df_hechos.drop(columns = columnas_borrar, axis=1)


In [ ]:
df_hechos.info()

In [110]:

# Utilizo una función para convertir diferentes tipos de entradas a un objeto de tiempo

def convertir_tiempo(x):
    # Verificamos si la entrada es una cadena de texto
    if isinstance(x, str):
        try:
            # Intentamos convertir la cadena de texto a un objeto de tiempo y lo retornamos
            return datetime.strptime(x, "%H:%M:%S").time()
        except ValueError:
            # Si hay un error en la conversión, retornamos None
            return None
    # Verificamos si la entrada es un objeto datetime
    elif isinstance(x, datetime):
        # Convertimos el objeto datetime a su correspondiente objeto de tiempo y lo retornamos
        return x.time()
    # Si la entrada no es ni cadena de texto ni datetime, retornamos la entrada sin modificar
    return x

In [111]:
# Cambiamos el tipo de dato de la columna 'Hora' aplicando la función convertir_tiempo a cada elemento.
# Esta función ajusta los valores a un formato de tiempo estandarizado o retorna None si la conversión no es posible.
df_hechos['HORA'] = df_hechos['HORA'].apply(lambda x: convertir_tiempo(x))

In [ ]:
# Imprimo y buscamos registros donde la columna 'Hora' es de tipo NoneType,
# lo que indica que la conversión no fue posible para esos registros.
print('Registro con NoneType:')
df_hechos[df_hechos['HORA'].isna()]

# Ya no hay registros raros

In [113]:
# saco la moda de la columna para saber cual es la hora mas comun y rellenan

# calcula la hora mas común
hora_moda = df_hechos['HORA'].mode().iloc[0]
print(f'La hora mas común es: {hora_moda}')


La hora mas común es: 09:00:00


In [114]:
# Reemplazo el valor None por la hora más común
df_hechos['HORA'].fillna(hora_moda, inplace=True)


In [ ]:
# Ahora si trato de cambiar el tipo de dato otra vez

# df_hechos['HORA'] = pd.to_datetime(df_hechos['HORA']).dt.time

# Para dejarla como fecha la tengo que juntar con la fecha y pued prefiero entonces dejarla en objeto


In [ ]:
df_hechos.head()

In [116]:
# Se imputa la hora moda al dato faltante
df_hechos['HORA'] = df_hechos['HORA'].apply(lambda x: int(hora_moda.hour) if x == "SD" else x)
# Se verifica el tipo de dato
df_hechos['HORA'].apply(type).value_counts()

HORA
<class 'datetime.time'>    696
Name: count, dtype: int64

In [ ]:
df_hechos.head()

In [118]:
# Se reemplaza el valor None por la hora más común
df_hechos['HORA'].fillna(hora_moda, inplace=True)

# se verifica el tipo de dato para la columna
df_hechos['HORA'].apply(type).value_counts()

HORA
<class 'datetime.time'>    696
Name: count, dtype: int64

Ya me ocupe de la fecha, ahora revisaré todas las columnas de ubicación 

In [17]:
# Averiguando en Chatgpt veo la función para devolver la dirección con las coordenadas
# para eso tomo una fila y hago el ejemplo

In [119]:
def obtener_direccion(COORDENADAS):
    latitud, longitud = COORDENADAS
    geolocalizador = Nominatim(user_agent="mi_aplicacion")
    ubicacion = geolocalizador.reverse(COORDENADAS, language='es')
    return ubicacion.address

In [120]:
# tomo ejemplo de Coordenadas geográficas, suponiendo que posx es latitud y posy altitud
LATITUD = -34.68757022
LONGITUD = -58.47533969

In [121]:
obtener_direccion((-34.68757022, -58.47533969))

'6502, Avenida General Francisco Fernández de La Cruz, Villa Riachuelo, Buenos Aires, Distrito del Deporte, Comuna 8, Ciudad Autónoma de Buenos Aires, C1439COV, Argentina'

In [78]:
# En realidad funcionó!!!, la dirección que me da el sistema es
# Dirección: 6502, Avenida General Francisco Fernández de La Cruz, Villa Riachuelo, Buenos Aires, Distrito del Deporte, Comuna 8, Ciudad Autónoma de Buenos Aires, C1439COV, Argentina
# Tambien cuadra numero de la comuna

In [122]:
# ya sabiendo que si son las coordenadas, Cambio el nombre de las columnas por latitud y longitud, las junto en una que se 
# llame coordenadas y con esa columna ya tendría toda la información

df_hechos = df_hechos.rename(columns={'pos x': 'LONGITUD'})
df_hechos = df_hechos.rename(columns={'pos y': 'LATITUD'})


In [ ]:
df_hechos.head()

In [124]:
df_hechos.size

11136

In [125]:
# Junto las 2 columnas en una que se llame COORDENADAS, por que la función que encontré solo tiene un argumento

df_hechos['COORDENADAS'] = df_hechos.apply(lambda row: (row['LONGITUD'], row['LATITUD']), axis=1)


In [ ]:
df_hechos.head()

In [127]:
# Con las coordenada ya tengo toda la info y puedo graficar en powebi asi que borro las que no interesan tanto

Columnas_borrar = ['TIPO_DE_CALLE', 'Calle',  'XY (CABA)', 'LONGITUD', 'LUGAR_DEL_HECHO', 'LATITUD', 'PARTICIPANTES']


In [128]:
df_hechos =  df_hechos.drop(Columnas_borrar, axis=1)


In [ ]:
df_hechos.head()

In [ ]:
df_hechos.info()

In [131]:
# Para que haga merge cambio el nombre id

df_hechos = df_hechos.rename(columns={'ID': 'ID_hecho'})

In [133]:
# Voy a renombrar Cruce y Dirección Normalizada por que todo esta en mayusculas asi que no se ve bien

df_hechos = df_hechos.rename(columns={'Cruce': 'CRUCE'})
df_hechos = df_hechos.rename(columns={'Dirección Normalizada': 'DIRECCION'})


In [ ]:
df_hechos.head()

In [135]:
# Borro nulos en dirección

df_hechos = df_hechos.dropna(subset=['DIRECCION'])


In [ ]:
# Confirmo que los datos están listos para ser analizados en un eda, grabo el dataframe df_hechos 
#Pero no me dejo grabarlo por que indicaba que no entendía el tipo de bojeto coordenada asi que tocó 
# Define una función para convertir las cadenas de texto en tuplas de números


In [137]:
df_hechos.head()

,ID_hecho,N_VICTIMAS,FECHA,HORA,DIRECCION,COMUNA,VICTIMA,ACUSADO,COORDENADAS
0,2016-0001,1,2016-01-01,04:00:00,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,MOTO,AUTO,"(-58.47533969, -34.68757022)"
1,2016-0002,1,2016-01-02,01:15:00,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,AUTO,PASAJEROS,"(-58.50877521, -34.66977709)"
2,2016-0003,1,2016-01-03,07:00:00,ENTRE RIOS AV. 2034,1,MOTO,AUTO,"(-58.39040293, -34.63189362)"
3,2016-0004,1,2016-01-10,00:00:00,"LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,MOTO,SD,"(-58.46503904, -34.68092974)"
4,2016-0005,1,2016-01-21,05:20:00,"SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,MOTO,PASAJEROS,"(-58.38718297, -34.62246630)"


In [ ]:
df_hechos.info()

In [143]:
df_hechos.to_parquet('df_hechos.parquet')
